# Lab 3 - Math 178, Winter 2025

You are encouraged to work in groups of up to 3 total students, but each student should submit their own file. (It's fine for everyone in the group to submit the same file.)

Put the full names of everyone in your group :)

**Names**:

In the begeining of the document: please introduce the data set and the goal of your project. 
for example 
* Dataset: `world_cup22.csv`
* Goal: Can we use statistics from a team's previous match to predict how many goals they will score in the World Cup?



## Data Analysis
In this section, you will show how you analyzed the dataset and identified the goal of your project.

* Read in the attached `world_cup22.csv` file and store it as a pandas DataFrame.

In [51]:
# Imort the libraries in the begining of the code
import pandas as pd

Do exploratory data analysis and write down one discovery of `world_cup22.csv`.

In [2]:
#This is the sandard practice of reading a csv file in pandas 
df = pd.read_csv("world_cup22.csv")

In [ ]:
# This is not necessary but it is a good practice to check the data types of the columns
df.dtypes

In [ ]:
df.info()

In [ ]:
df.shape

## Prepare the data

* Rescale every column from "month" and beyond (i.e., every numeric column except for the "number of goals" column) to have mean (approximately) zero and standard deviation (approximately) one.

The most straightforward way to achieve this in my opinion is with the following code.  You should replace `???` here with code that rescales the pandas Series `col`.

```
df.loc[:, "month":] = df.loc[:, "month":].apply(lambda col: ???, axis=0)
```

* Use `train_test_split` to divide the data into a training set of length `60` a test set of length `36`.  Use the `random_state` keyword argument so that you get consistent results.  Here `X_train` and `X_test` should be DataFrames with all of the rescaled numeric columns.  `y_train` and `y_test` should be pandas Series containing only the "number of goals" column.  (We will never use the "team" column in this lab; it's just there for your own interest, and it was crucial to preparing the data, because we needed the statistics from the team's previous match.)

## First Model:
Upon selection of the models, we need to explain and answer the following questions in the final project:


1. The model we choose is: 
2. What do we expect this model to behave: (over fit, underfit, large error, etc.)
3. Why we choose this model and why do we expect such behavior? 

In this lab, the first model we will use is linear regression. Answer the following questions in markdown:

* The model we choose is:
* What do we expect this model to behave:

## Evaluate the First Model

### Description of Evaluation Tool and Results

For the evaluation of our first model, we used the Mean Squared Error (MSE) metric. The MSE is a common measure of the quality of a regression model; it represents the average squared difference between the observed actual outcomes and the outcomes predicted by the model.

### Training and Test MSE
In this lab, let's answer the following questions:
* What is the training Mean Squared Error?
* What is the test Mean Squared Error?
* Why does this suggest overfitting?


## Aside: coefficient magnitudes

Execute the following code to get a sense for which coefficients were deemed most important in our linear regression model.  (The following code assumes `reg` is the name of your fit `LinearRegression` object. 

```
import altair as alt

df_coef = pd.DataFrame({
    'feature': reg.feature_names_in_,
    'coef': reg.coef_
})

chart = alt.Chart(df_coef).mark_bar().encode(
    x='coef',
    y=alt.Y('feature').sort('x') # delete `sort('x')` if necessary
).properties(
    title='Coefficients'
)
chart
```

Comment: In my preparation for this lab, these coefficient sizes have been very unstable.  That is another sign of overfitting.  So if you do this twice with different random states, do not expect similar results.

Comment 2: These coefficient sizes would not be meaningful (in relation to each other) if we had not rescaled the input columns to have equal standard deviations.

## Second method: 

Pick one option to complete: cross-validation or bootstrapping. Explain the motivation.

## Select your second method and state motivation
We saw above that using every column led to overfitting.  Here we will restrict ourselves to only using three columns as predictors.  But which three columns should we use?

* Create a list of all possible length-3 tuples of column names from `X_train`.  (Be sure your triples contain strings, not pandas Series.  We want triples of column names, not triples of columns.)

Comment.  I did this using the `combinations` function from the `itertools` module, and converting the resulting generator into a list (just by wrapping it in the `list` function).

Comment 2.  You would not want to do this for all possible length-10 tuples, because there would be too many.  In this case, our list will have length 14,190, which is no concern.

In [15]:
from itertools import combinations

In [16]:
triple_list = list(combinations(X_train.columns, 3))

In [ ]:
len(triple_list)

* Choose 1,000 of these triples randomly using `rng.choice` where `rng` is a NumPy `default_rng` object; store the resulting NumPy array using the variable name `random_tuples`.  Use a `seed` keyword argument to `default_rng` so that you get reproducible results.  I recommend instantiating `rng` and calling `rng.choice` in the same cell, because this will help reproducibility.

Comment.  If you wanted to have for example length-10 tuples, then you should just do this step directly, without ever using `itertools`.

## Cross validation

Overview: For each triple of features in `random_tuples`, we will get an estimated Mean Squared Error using 10-fold cross-validation.  We will choose as columns those which produce the best (i.e., lowest) MSE.

Use scikit-learn's `cross_validate` function to generate a list as follows.
* Each entry in the list will be a length-2 tuple consisting of 1st the columns and 2nd the MSE cross-validation score.
* Specify to use 10-fold cross-validation using the `cv` keyword argument.
* Specify `"neg_mean_squared_error"` as the `scoring` keyword argument to `cross_validate`.
* Do not use the full `X_train` in `cross_validate`.  Instead only use the three columns in `triple`.
* Compute the mean of the resulting `"test_score"`.  This will be the negative of the mean of the MSEs, so negate it to get a traditional (positive) MSE.

```
mse_list = []

for triple in random_tuples:
    cv_results = cross_validate(???)
    cv_mse = ???
    mse_list.append((tuple(triple), cv_mse))
```

Comment.  This code took about two minutes to run when I tried it.  If necessary you can decrease the number of triples used.

Comment 2.  The `tuple(triple)` is there to convert `triple` from a NumPy array into a `tuple`, so that it is hashable and is thus allowed to serve as a key to a dictionary.

Comment 3.  Reality check: when I ran the code, these were the first two elements of `mse_list`.  The triples represent the columns used and the numbers represent the corresponding cross-validation Mean Squared Errors.
```
[(('central channel', 'inbetween offers to receive', 'red cards'),
  1.228113034518691),
 (('completed defensive line breaks', 'passes', 'crosses'),
  1.3288270772012454)]
```



In [20]:
from sklearn.model_selection import cross_validate

* Convert `mse_list` to a dictionary and then to a pandas Series.  (It did not work when I tried to immediately convert it to a dictionary.)  Name the result `mse_series`.

* What triple of columns (from the thousand we tested) produces the lowest cross-validation MSE?  This is surprisingly easy to answer: use the `idxmin` method of a pandas Series.  What is the MSE in that case?

In [ ]:
mse_series.idxmin()

In [ ]:
mse_series.min()

## Bootstrap MSE Estimation 
The goal of this section is to implement **bootstrapping** to estimate the **Mean Squared Error (MSE)** of different subsets of features in a regression model. Instead of using traditional k-fold cross-validation, you will use bootstrapping to repeatedly sample from the training data (with replacement) to assess model performance. In this project, you'll be working with the `world_cup22.csv` dataset.

---
### 1. Load the Required Libraries
Ensure you have the necessary Python libraries installed. You will use:
- `numpy` for numerical operations.
- `sklearn.utils.resample` for bootstrapping.
- `sklearn.linear_model.LinearRegression` for building a regression model.
- `sklearn.metrics.mean_squared_error` for calculating MSE.




---

### 2. Define the Bootstrapping Process

For each feature combination (referred to as `triple`):
- **Extract** the corresponding three-column subset from `X_train`.
- **Set** `y_train` as the target variable.
- **Perform** a fixed number of bootstrap iterations (e.g., 1000).

Within each bootstrap iteration:
- **Sample** (with replacement) from `X_train` and `y_train` to create a new training set.
- **Train** a `LinearRegression` model on the resampled data.
- **Predict** on the original subset (`X_subset`).
- **Compute** and store the MSE.

The final MSE for each feature triple is the average of the bootstrap MSE values.




---

### 3. Implement the Bootstrapping Loop

Iterate over `random_tuples` (each representing a set of three features) and store the results in `mse_list`. Each entry in `mse_list` should be a tuple containing:
- The feature set (as a tuple)
- The corresponding average MSE from bootstrapping

Below is an example code snippet implementing this process:

In [ ]:
mse_list = []
num_bootstrap_samples = 10 # Adjust this number for performance if necessary

for triple in random_tuples:
    # Use only the three selected columns from X_train for this feature combination
    X_subset = X_train[list(triple)]
    y = y_train  # Target variable

    mse_bootstrap = []

    for _ in range(num_bootstrap_samples):
        # Bootstrap sampling: sample with replacement from X_subset and y
        X_resampled, y_resampled = resample(X_subset, y, replace=True)

        # Train a linear regression model on the resampled data
        model = LinearRegression()
        model.fit(X_resampled, y_resampled)

        # Predict on the original subset
        y_pred = model.predict(X_subset)

        # Compute and store the Mean Squared Error
        mse_bootstrap.append(mean_squared_error(y, y_pred))
    
    # Append the feature triple and the average bootstrap MSE to the results list
    mse_list.append((tuple(triple), np.mean(mse_bootstrap)))



---

### 4. Expected Output Format

Your final output, `mse_list`, should be a list of tuples. Each tuple should contain:
1. The feature set (as a tuple)
2. The corresponding average MSE from bootstrapping

**Example:**

```python
[(('hour', 'day', 'month'), 1.248),
 (('possession', 'possession in contest', 'total attempts'), 1.325),
 (('left channel', 'left inside channel', 'central channel'), 1.189)]



# Guided Summary & Reflection

After completing the bootstrap MSE estimation on the `world_cup22.csv` dataset, please reflect on the following questions. Write your reflections in markdown cells in your notebook.


### Reflection Questions:
1. **Why do we need to use cross-validation/bootstrapping?** 

2. **What did we find?**  

3. **What can we say about this project?**  

---